In [32]:
import os
from dotenv import load_dotenv
import openai

In [33]:
# Load environment variables from .env file
load_dotenv()

True

In [36]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [37]:
from llama_index.core import SimpleDirectoryReader

In [47]:
documents = SimpleDirectoryReader(
    # input_files = ["./docs/colbert.pdf"]
    input_files = ["./docs/PLAID.pdf", "./docs/colbert.pdf", "./docs/colbert-v2.pdf"]
).load_data()

In [48]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

40 

<class 'llama_index.core.schema.Document'>
Doc ID: 1d86c135-0772-49a6-b01e-306abc1aa60d
Text: PLAID: An Efficient Engine for Late Interaction Retrieval Keshav
Santhanam∗ keshav2@stanford.edu Stanford University United StatesOmar
Khattab∗ okhattab@stanford.edu Stanford University United States
Christopher Potts Stanford University United StatesMatei Zaharia
Stanford University United States ABSTRACT Pre-trained language models
are increas...


In [49]:
# print all documents length with a delay i.e. clear the screen after printing each document
# import time
# for doc in documents:
#     print(doc)
#     time.sleep(1)
#     os.system('clear')

In [50]:
len(documents)

40

In [51]:
from llama_index.core import Document

In [52]:
document = Document(text="\n\n".join([doc.text for doc in documents]))

In [53]:
print(document)

Doc ID: 58053497-9aea-42bd-9aa3-db0f570f0097
Text: PLAID: An Efficient Engine for Late Interaction Retrieval Keshav
Santhanam∗ keshav2@stanford.edu Stanford University United StatesOmar
Khattab∗ okhattab@stanford.edu Stanford University United States
Christopher Potts Stanford University United StatesMatei Zaharia
Stanford University United States ABSTRACT Pre-trained language models
are increas...


In [54]:
from llama_index.core import VectorStoreIndex
from llama_index.core import ServiceContext
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)

/var/folders/0j/d0xd373d2tb7zdzcsqrk3mpw0000gn/T/ipykernel_24193/339795532.py:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [56]:
index = VectorStoreIndex.from_documents(
    [document],
    service_context=service_context
    )

In [57]:
query_engine = index.as_query_engine()

In [58]:
response = query_engine.query(
    "explain the query and document encoder. Then show differences between them"
)
print(str(response))

The query encoder preprocesses textual queries by tokenizing them into BERT-based WordPiece tokens and prepending a special token [Q] to indicate it is a query. The tokenized query is then passed through BERT's deep transformer architecture to compute contextualized representations of each token. Query augmentation is applied by padding the query with BERT's special [mask] tokens to allow for learning to expand or re-weigh terms based on importance. The output embeddings are passed through a linear layer for dimension control and normalization.

On the other hand, the document encoder segments a document into tokens and prepends BERT's start token [CLS] followed by a special token [D] to indicate it is a document. Unlike queries, no [mask] tokens are appended to documents. After processing through BERT and a linear layer, the document encoder filters out embeddings corresponding to punctuation symbols. This filtering step aims to reduce the number of embeddings per document by excludin

In [59]:
response = query_engine.query(
    "explain difference between query and document encoder visually"
)

In [60]:
print(str(response))

The query encoder preprocesses textual queries by adding a special token [Q] at the beginning and padding with [mask] tokens if needed. It then passes the sequence through BERT and a linear layer. On the other hand, the document encoder segments documents into tokens, adds a start token [CLS] followed by [D], and filters out embeddings corresponding to punctuation symbols. The document encoder does not use [mask] tokens like the query encoder. Both encoders ultimately produce bags of embeddings for queries and documents, respectively.
